In [12]:
# ignore this cell, this is just a helper cell to provide the magic %highlight_file
%run ../highlighter.py

# Configure

This is a runbook to configure the network. To do so we are going to load first some data from the directory `../extra_data/` and then a bunch of templates to generate, based on that extra data, the configuration for the devices.

## Extra data

Let's first look at the extra data we are going to use:

In [5]:
%ls ../extra_data/*

../extra_data/leaf00.cmh:
l3.yaml

../extra_data/leaf01.cmh:
l3.yaml

../extra_data/spine00.cmh:
l3.yaml

../extra_data/spine01.cmh:
l3.yaml


Now let's look at one of the files for reference:

In [7]:
% cat ../extra_data/leaf00.cmh/l3.yaml

---
interfaces:
  Ethernet1:
    connects_to: spine00.cmh
    ipv4: 10.0.0.1/31
    enabled: false
  Ethernet2:
    connects_to: spine01.cmh
    ipv4: 10.0.1.1/31
    enabled: true

sessions:
  - ipv4: 10.0.0.0
    peer_as: 65000
  - ipv4: 10.0.1.0
    peer_as: 65000


## Templates

To configure the network we will transform the data we saw before into actual configurationusing jinja2 templates. The templates are:

In [8]:
%ls ../templates/*

../templates/eos:
base.j2        interfaces.j2  leaf.j2        routing.j2     spine.j2

../templates/junos:
base.j2        interfaces.j2  leaf.j2        routing.j2     spine.j2


As an example, let's look how the ``interfaces.j2`` template will consume the extra data we saw before to configure the interfaces:

In [10]:
%cat ../templates/eos/interfaces.j2

{% for interface, data in l3.interfaces.items() %}
interface {{ interface }}
   no switchport
   ip address {{ data.ipv4 }}
   description link to {{ data.connects_to }}
   {{ "no" if data.enabled else "" }} shutdown
{% endfor %}



## Code

Now let's look at the code that will sticth everything together:

In [11]:
%highlight_file configure.py

## Demo

Finally let's see everything in action:

In [14]:
%run configure.py

**** Playbook to configure the network *****************************************
* spine00.cmh ** changed : True ************************************************
---- Loading Configuration on the device ** changed : True  --------------------
@@ -8,7 +8,8 @@
 !
 transceiver qsfp default-mode 4x10G
 !
-hostname localhost
+hostname spine00.cmh
+ip domain-name cmh.acme.com
 !
 spanning-tree mode mstp
 !
@@ -20,13 +21,28 @@
 username vagrant privilege 15 role network-admin secret sha512 $6$kRQZJTqx69hOW5ag$Y6VX8Kk37TWEsriKdr6ixqvMuUSSbuFu2Eh/5SIet2TCeXP3bdlwikIAruPp6lHB5HdC.t6tPsZVctHMU7H590
 !
 interface Ethernet1
+   description link to leaf00.cmh
+   no switchport
+   ip address 10.0.0.0/31
 !
 interface Ethernet2
+   description link to leaf01.cmh
+   no switchport
+   ip address 10.0.0.2/31
 !
 interface Management1
    ip address 10.0.2.15/24
 !
-no ip routing
+ip routing
+!
+router bgp 65000
+   neighbor 10.0.0.1 remote-as 65100
+   neighbor 10.0.0.1 maximum-routes 12000 
+   neighb

In [15]:
%run configure.py

**** Playbook to configure the network *****************************************
* spine00.cmh ** changed : False ***********************************************
---- Loading Configuration on the device ** changed : False  -------------------

* spine01.cmh ** changed : False ***********************************************
---- Loading Configuration on the device ** changed : False  -------------------

* leaf00.cmh ** changed : False ************************************************
---- Loading Configuration on the device ** changed : False  -------------------

* leaf01.cmh ** changed : False ************************************************
---- Loading Configuration on the device ** changed : False  -------------------



The tool also detects unwanted changes and corrects them. For instance, let's change the hostname manually:

    spine00.cmh((config)#hostname localhost
    localhost(config)#

And run the runbook again:

In [16]:
%run configure.py

**** Playbook to configure the network *****************************************
* spine00.cmh ** changed : True ************************************************
---- Loading Configuration on the device ** changed : True  --------------------
@@ -8,7 +8,7 @@
 !
 transceiver qsfp default-mode 4x10G
 !
-hostname localhost
+hostname spine00.cmh
 ip domain-name cmh.acme.com
 !
 spanning-tree mode mstp

* spine01.cmh ** changed : False ***********************************************
---- Loading Configuration on the device ** changed : False  -------------------

* leaf00.cmh ** changed : False ************************************************
---- Loading Configuration on the device ** changed : False  -------------------

* leaf01.cmh ** changed : False ************************************************
---- Loading Configuration on the device ** changed : False  -------------------

